In [1]:
# import tensorflow as tf
# print(tf.__version__)  # Should output without errors


In [2]:
# !pip install tensorflow==2.13
# !pip install tensorflow-gpu


In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [4]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def lstm_regression_lags_only(
    series,
    district_name,
    max_lags=12,
    differencing=True,
    epochs=100,
    batch_size=32,
    verbose=0
):
    """
    Robust LSTM forecasting with full error handling
    Returns (None, None) on error to maintain pipeline flow
    """
    try:
        # 0. Initial checks
        if len(series) < max_lags + 24:  # Minimum 24 points after lag creation
            raise ValueError(f"Series too short ({len(series)} points) for {max_lags} lags")

        # 1. Directory setup
        os.makedirs('LSTM_Forecasts', exist_ok=True)
        
        # 2. Stationarity handling
        original_series = series.copy()
        d = 0
        if differencing:
            adf_result = adfuller(series.dropna())
            if adf_result[1] > 0.05:
                d = 1
                series = series.diff().dropna()
                if len(series) < 10:
                    raise ValueError("Insufficient data after differencing")

        # 3. Create lag features
        df = pd.DataFrame({'y': series})
        for lag in range(1, max_lags + 1):
            df[f'lag_{lag}'] = df['y'].shift(lag)
        df = df.dropna()

        # 4. Temporal split with index preservation
        train_size = int(len(df) * 0.8)
        if train_size < 10 or (len(df) - train_size) < 2:
            raise ValueError("Insufficient train/test split")
            
        train = df.iloc[:train_size]
        test = df.iloc[train_size:]
        test_series = test['y']  # Preserve for index

        # 5. Prepare numpy arrays
        X_train = train.drop(columns=['y']).values
        y_train = train['y'].values.reshape(-1, 1)
        X_test = test.drop(columns=['y']).values
        y_test_np = test['y'].values.reshape(-1, 1)

        # 6. Scaling
        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()

        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)
        y_train_scaled = scaler_y.fit_transform(y_train)
        y_test_scaled = scaler_y.transform(y_test_np)

        # 7. Reshape for LSTM [samples, timesteps, features]
        X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
        X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

        # 8. Build model
        model = Sequential()
        model.add(LSTM(50, activation='relu', input_shape=(1, max_lags)))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')

        # 9. Train with validation
        history = model.fit(
            X_train_scaled, 
            y_train_scaled,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_split=0.2,
            shuffle=False
        )

        # 10. Predict and inverse scaling
        y_pred_scaled = model.predict(X_test_scaled, verbose=verbose)
        y_pred = scaler_y.inverse_transform(y_pred_scaled).flatten()

        # 11. Inverse differencing
        if d == 1:
            last_train_value = original_series.iloc[train.index[0] - 1]
            y_pred = np.cumsum(y_pred) + last_train_value
            y_test_values = original_series.iloc[test.index].values
        else:
            y_test_values = test_series.values

        # 12. Create output (preserve original indices)
        forecast_df = pd.DataFrame({
            'district': district_name,
            'date': test_series.index,
            'actual': y_test_values,
            'forecast': y_pred
        })

        # 13. Calculate metrics
        rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast']))

        # 14. Save results
        forecast_df.to_csv(
            os.path.join('LSTM_Forecasts', f'{district_name}_forecast.csv'), 
            index=False
        )

        # 15. Plotting
        plt.figure(figsize=(12, 6))
        plt.plot(original_series.index, original_series, label='Original')
        plt.plot(forecast_df['date'], forecast_df['forecast'], label='LSTM Forecast')
        plt.title(f'{district_name} Forecast (RMSE: {rmse:.2f})')
        plt.legend()
        plt.savefig(os.path.join('LSTM_Forecasts', f'{district_name}_forecast_plot.png'))
        plt.close()

        return {
            'district': district_name,
            'rmse': rmse,
            'epochs': epochs,
            'batch_size': batch_size,
            'training_loss': history.history['loss'],
            'validation_loss': history.history['val_loss']
        }, forecast_df

    except Exception as e:
        print(f"⚠️ Skipping {district_name}: {str(e)}")
        return None, None


In [6]:
districts = data['district'].unique()
rmse_values = []

# Usage Example


def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I48"].asfreq('MS')
        
        results, forecast = results, forecast = lstm_regression_lags_only(
            series=ts,
            district_name=district,
            max_lags=6,
            epochs=150,
            batch_size=64
        )

        
    
        # Show results
        print("=== Metrics ===")
        print(results)
        print("\n=== Forecast Data ===")
        print(forecast)
    
    return results
run_for_each_district()

=== Metrics ===
{'district': 'AHMEDNAGAR', 'rmse': 64.80708610156742, 'epochs': 150, 'batch_size': 64, 'training_loss': [0.3599340617656708, 0.3525361716747284, 0.3452571630477905, 0.338100403547287, 0.33106476068496704, 0.32413727045059204, 0.3173350989818573, 0.3106363117694855, 0.304063618183136, 0.2976279854774475, 0.2912938594818115, 0.28504830598831177, 0.2788994014263153, 0.2728445827960968, 0.2668590843677521, 0.2609609067440033, 0.2551329433917999, 0.24937711656093597, 0.24370266497135162, 0.23812507092952728, 0.2326185405254364, 0.2271588146686554, 0.22174951434135437, 0.21643076837062836, 0.21118861436843872, 0.20603033900260925, 0.20092903077602386, 0.19589903950691223, 0.19093656539916992, 0.18603914976119995, 0.18119151890277863, 0.1764000952243805, 0.17169855535030365, 0.16707253456115723, 0.16251584887504578, 0.1580384075641632, 0.1536436825990677, 0.14932860434055328, 0.1450965255498886, 0.14095383882522583, 0.13689793646335602, 0.1329343318939209, 0.12906570732593536,